<a href="https://colab.research.google.com/github/tasnia87/Bangla-Sign-Language/blob/master/cnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pickle
import cv2, os
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import backend as K

In [ ]:
K.set_image_dim_ordering('tf')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def get_image_size():
	img = cv2.imread('gestures/0/1.JPG', 0)
	return img.shape

def get_num_of_classes():
	return len(os.listdir('gestures/'))

image_x, image_y = get_image_size()

In [ ]:
def cnn_model():
	num_of_classes = get_num_of_classes()
	model = Sequential()
	model.add(Conv2D(32, (5,5), input_shape=(image_x, image_y, 1), activation='sigmoid'))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
	model.add(Conv2D(64, (5,5), activation='sigmoid'))
	model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
	model.add(Flatten())
	model.add(Dense(1024, activation='relu'))
	model.add(Dropout(0.6))
	model.add(Dense(num_of_classes, activation='softmax'))
	sgd = optimizers.SGD(lr=1e-2)
	model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
	filepath="cnn_model_keras2.h5"
	checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
	#checkpoint2 = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
	callbacks_list = [checkpoint1]
	return model, callbacks_list

In [ ]:
def train():
	with open("train_images", "rb") as f:
		train_images = np.array(pickle.load(f))
	with open("train_labels", "rb") as f:
		train_labels = np.array(pickle.load(f), dtype=np.int32)

	with open("test_images", "rb") as f:
		test_images = np.array(pickle.load(f))
	with open("test_labels", "rb") as f:
		test_labels = np.array(pickle.load(f), dtype=np.int32)

	train_images = np.reshape(train_images, (train_images.shape[0], image_x, image_y, 1))
	test_images = np.reshape(test_images, (test_images.shape[0], image_x, image_y, 1))
	train_labels = np_utils.to_categorical(train_labels)
	test_labels = np_utils.to_categorical(test_labels)

	model, callbacks_list = cnn_model()
	model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=500, batch_size=100, callbacks=callbacks_list)
	scores = model.evaluate(test_images, test_labels, verbose=0)
	print("CNN Error: %.2f%%" % (100-scores[1]*100))
	#model.save('cnn_model_keras2.h5')

In [ ]:
train()
K.clear_session();

Train on 833 samples, validate on 167 samples
Epoch 1/500
833/833 [==============================] - 22s 27ms/step - loss: 2.9328 - acc: 0.1188 - val_loss: 2.3755 - val_acc: 0.1018

Epoch 00001: val_acc improved from -inf to 0.10180, saving model to cnn_model_keras2.h5
Epoch 2/500
833/833 [==============================] - 21s 25ms/step - loss: 2.4077 - acc: 0.1176 - val_loss: 2.3136 - val_acc: 0.1018

Epoch 00002: val_acc improved from 0.10180 to 0.10180, saving model to cnn_model_keras2.h5
Epoch 3/500
833/833 [==============================] - 21s 26ms/step - loss: 2.3202 - acc: 0.1273 - val_loss: 2.2186 - val_acc: 0.1198

Epoch 00003: val_acc improved from 0.10180 to 0.11976, saving model to cnn_model_keras2.h5
Epoch 4/500
833/833 [==============================] - 20s 24ms/step - loss: 2.2326 - acc: 0.1861 - val_loss: 2.2621 - val_acc: 0.1437

Epoch 00004: val_acc improved from 0.11976 to 0.14371, saving model to cnn_model_keras2.h5
Epoch 5/500
833/833 [============================

In [ ]:
import tensorflow
print(tensorflow.__version__)

1.8.0
